In [ ]:
import os
import sys

import cv2
import matplotlib.pyplot as plt
import numpy as np

sys.path.append("../")
from src.adnet import compute_landmarks

%load_ext autoreload
%autoreload 2

In [ ]:
img_p = "../data/c-07-twofaces_cropped.png"
model_p = "../checkpoints/adnet/adnet_ofiq.onnx"
landmarks = compute_landmarks(img_p, model_p)
# print(landmarks.shape)

img = cv2.imread(img_p)
for idx in range(len(landmarks)):
    x, y = landmarks[idx]
    cv2.circle(img, (x, y), 3, (255, 0, 0), cv2.FILLED)
    cv2.putText(img, str(idx), (x + 2, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()
